# SQL and Database

What is a database? 
- A database is a collection of information that is organized so that it can be easily accessed, managed, and updated.
- In a relational database, digital information about a specific customer is organized into rows, columns, and tables which are indexed to make it easier to find relevant information through SQL queries.
- Source: https://searchsqlserver.techtarget.com/definition/database


Why create a relational database?
- Flexible, can add new data and new relationships easily
- Eliminates redundancies (e.g., a csv with several rows of repeated data)
- Source: https://www.ibm.com/cloud/learn/relational-databases
- Example: https://www.pragimtech.com/blog/contribute/article_images/2220211210231003/what-is-a-relational-database.jpg


What is ORM? 
- ORM stands for Object Relational Mapping. 
- ORM is a technique used in creating a "bridge" between OOP and relational databases. 
- An ORM tool is a software designed to help OOP (like Python) to interact with relational databases. 
<br>
<br>

#### Python offers several ORM tools that help us to do that! 

Here is an example of SQL code that retrieves information about a particular user from a database: 

`SELECT id, name, email, country, phone_number FROM users WHERE id = 20`

This code returns specified information about this user from a table called "users" using the WHERE clause where we specified that id = 20 for this user. 

On the other hand, an ORM tool can do the exact same query with much shorter code! 

`users.GetById(20)`

<br>
<br>
Let's install `sqlite` first! 

- On Mac, open terminal, then run `brew install sqlite3`
- On Windows, follow this tutorial: https://www.guru99.com/download-install-sqlite.html

Now we can install this library `sqlalchemy` for python. 

- Check tutorial: http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/
- Find documentation: https://www.kite.com/python/docs/

<br>
<br>

In [1]:
# !pip3 install sqlalchemy
import sqlalchemy

import warnings
warnings.filterwarnings('ignore')

##### 1. Establish connectivity using the Engine

The engine is typically a global object created using `create_engine()` just once for a particular database. 

In [2]:
# Make sure to change to your directory
engine = sqlalchemy.create_engine('sqlite:///players.db')

The parameter `echo = True` makes SQLAlchemy log all SQL commands it is doing while you apply commands. 

For now, I'm just turning it off to make the cell output cleaner. 

The database will be intact, until you execute the commands. 

##### 2. Instantiate a Base

With the help of ORM tools, we don't need to write SQL queries anymore. 

Everything is done using a declarative system. 

The `Base` class maintains a catalog of all the tables and classes (i.e. structures of the database). 

More information: https://stackoverflow.com/questions/1279613/what-is-an-orm-how-does-it-work-and-how-should-i-use-one

In [3]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base() 

In our database, each class is a table. Each attribute is a column in the table. 

A brief note on the table relations in SQL. 
1. One-to-one: one record of a table is linked to zero or one record of another table. 
    - A student can only have one student ID and one grade for a particular course. 
2. One-to-many: one record of a table can be mapped to multiple rows of another table. 
    - A student can have multiple emails of different type, e.g. school email, personal email, etc. 
3. Many-to-many: each row in a table can map to multiple rows in another table and vice versa. 
    - A student can know how to use several programming languages like R, Python, Julia. 
    - And one programming language (e.g. Python) can be mapped to multiple students who have the same skills. 


<br>

#### 3. Create our tables

Let's create a one-to-many example:   
One basketball team can have many players in each of them, but one player can only play for one basketball team. 

We need: 
1. Two tables: one for players, and one for teams
2. Foreign key on players to reference unique team ID (primary key for teams)
3. relationship() method to specify 

In [4]:
from sqlalchemy import Column, Integer, String, ForeignKey
# Create a table for Players
# Our Player table has 4 columns: ID, Name, Number, and Team ID
class Player(Base):
    __tablename__ = 'players'

    ## At first, only specify data types for the columns
    ## primary_key is unique, non-nullable identifier for row
    ## Have an ID column because player attributes (name, etc) are not unique
    ## At least 1 primary_key per table
    id = Column(Integer, primary_key = True) 
    name = Column(String)
    number = Column(Integer)
    
    ## ForeignKey tells us we have a relationship with another table ("teams") by the ("id") variable
    ## This info constrained to only come from that table
    ## What we are referencing is usually the primary key for that table
    team_id = Column(Integer, ForeignKey("teams.id")) 

    ## Populate non-ID fields in the table through the constructor
    def __init__(self, name, number, team=None):
        self.name = name
        self.number = number
        self.team = team
      
    # Optional to define the representation (what gets returned when you enter this object)
    def __repr__(self):
        return "<Player('%s', '%s')>" % (self.name, self.number)

When creating the tables, we can add simple constraints. 

For example, in the `players` table we created above, we already set a constraint on the column `id`. 

We said that this column `id` should be part of the primary key for this table. 

- A primary key uniquely identified a row in a table.
- A foreign key is used to link two tables together by referencing the primary key of the related table. 
- A primary key cannot have a NULL value, where foreign keys can take NULL values. 

In [5]:
from sqlalchemy.orm import relationship
# Our Team table has 2 columns: ID and Name
class Team(Base):
  __tablename__ = "teams"
  
  id = Column(Integer, primary_key=True)
  name = Column(String)

  ## - relationship() tells us another table wants to reference us
  ## - now notice we use "Player" object syntax and "team" member variable syntax
  ## - Note: this is NOT a column
  ##          but we can call <team obj>.players
  ##          or <player obj>.team
  players = relationship("Player", backref="team")
  
  def __init__(self, name):
    self.name = name
  
  def __repr__(self):
    return "<team('%s')>" % (self.name)

#### 4. Issue commands to database

- First time create tables
- The MetaData is a registry which includes the ability to make schema commands to database.
- Our SQLite database does not actually have a players table present, so we use MetaData to issue the SQL "CREATE TABLE" command to the database for all tables that don’t yet exist.

In [6]:
Base.metadata.create_all(engine)

Now we should see our database "player.db" in our working directory! 

Let's check what columns we have in the tables

In [7]:
# - SQLAlchemy represents info for specific table with Table object
Player.__table__  
# Team.__table__

Table('players', MetaData(), Column('id', Integer(), table=<players>, primary_key=True, nullable=False), Column('name', String(), table=<players>), Column('number', Integer(), table=<players>), Column('team_id', Integer(), ForeignKey('teams.id'), table=<players>), schema=None)

5. Create instances for each table

In [8]:
# Very similar syntax to what we've done before with class definitions!
# One instance for each table
p1 = Player(name = "Cecilia", number = 24)
t1 = Team(name = "WashU")

p1 and t1 are not yet connected 

In [9]:
t1.players

[]

In [10]:
# add team reference to player 
p1.team = t1
# now a part of team object
t1.players

[<Player('Cecilia', '24')>]

In [11]:
p1.team

<team('WashU')>

Let's create another Player Peter! 

In [12]:
peter = Player("Peter Bachman", 5)
# Let's print the ID
print(peter.id)

None


<br>
<br>

- Nothing?
- Even though we didn’t specify it in the constructor, the id attribute still produces a value of None when we access it (as opposed to Python’s usual behavior of raising AttributeError for an undefined attribute).
- When we put the `peter` object in the db a real ID will be assigned

6. Create a `Session` to actually store things into the database

A Session is a "holding zone" for all the objects and associations we have created. 
The Session will hold objects until we commit or close. 

In [13]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(engine) # sessionmake object
session = Session() # Session object

In [14]:
# add player
session.add(peter)
session.add(p1)

In [15]:
# add multiple players
session.add_all([Player("Irene", 7),
                 Player("Jie", 9),
                 Player("Leticia", 21),
                 Player("Tian", 33),
                 Player("Masanori", 19)])

In [16]:
# see what we've done this session
session.new 

IdentitySet([<Player('Peter Bachman', '5')>, <Player('Cecilia', '24')>, <team('WashU')>, <Player('Irene', '7')>, <Player('Jie', '9')>, <Player('Leticia', '21')>, <Player('Tian', '33')>, <Player('Masanori', '19')>])

In [17]:
# before we commit, if we made a mistake, we can correct it
# session.rollback() # it rolls back all changes

In [18]:
# now make changes to actual db
session.commit()

<br>
<br>

Now that we created the database, let's make some queries!

In [19]:
# Test again for ID... 
# (it keeps the count in the order they entered the database)
print('ID: ' + str(peter.id))
print('ID: ' + str(p1.id))

ID: 1
ID: 2


In [20]:
# Some querying
# order the results
# you can think of it as... session.query(TABLE).order_by(COLUMN)
for player in session.query(Player).order_by(Player.number): # .asc() .desc()
  print(player.number, player.name, player.id)

5 Peter Bachman 1
7 Irene 3
9 Jie 4
19 Masanori 7
21 Leticia 5
24 Cecilia 2
33 Tian 6


`.filter()`

Source: https://docs.sqlalchemy.org/en/20/tutorial/index.html#unified-tutorial

In [21]:
for player in session.query(Player).filter(Player.name == "Cecilia").order_by(Player.number):
  print(player.number, player.name)

24 Cecilia


In [22]:
for player in session.query(Player).filter(Player.name != "Cecilia").order_by(Player.number):
  print(player.number, player.name)

5 Peter Bachman
7 Irene
9 Jie
19 Masanori
21 Leticia
33 Tian


In [23]:
for player in session.query(Player).filter(Player.number > 10).order_by(Player.number):
  print(player.number, player.name)

19 Masanori
21 Leticia
24 Cecilia
33 Tian


`or_()`

In [24]:
from sqlalchemy import or_
for player in session.query(Player).filter(or_(Player.name == "Cecilia", Player.name == "Irene")).order_by(Player.number):
  print(player.number, player.name)

7 Irene
24 Cecilia


`.like()`

In [25]:
# return all the rows with 'name' column contains the partial string "Plum"
for player in session.query(Player).filter(Player.name.like("%ia%")).order_by(Player.number):
  print(player.number, player.name)

21 Leticia
24 Cecilia
33 Tian


`.and()`

In [26]:
from sqlalchemy import and_
for player in session.query(Player).filter(and_(Player.name.like("%ia%"), Player.number > 21)).order_by(Player.number):
  print(player.number, player.name)

24 Cecilia
33 Tian


In [27]:
# functions
# we can also apply functions using the func package
from sqlalchemy import func

In [28]:
# note that I need to query the columns I want back
for player in session.query(Player.name, Player.number, func.max(Player.number)):
  print(player.name, player.number)

Tian 33


In [29]:
# Results can be indexed as lists
results = session.query(Player).filter(and_(Player.name.like("%ia%"), Player.number > 10)).order_by(Player.number)
results.first()

<Player('Leticia', '21')>

In [30]:
# Check the number of elements in results
# len(results)
results.count()

3

In [31]:
results[0]
# results[1]
# results[2]

<Player('Leticia', '21')>

Let's create a new team, and assign some players to that team!

In [32]:
duke = Team('Duke')

In [33]:
# query all players
players = session.query(Player).all()
players

[<Player('Peter Bachman', '5')>,
 <Player('Cecilia', '24')>,
 <Player('Irene', '7')>,
 <Player('Jie', '9')>,
 <Player('Leticia', '21')>,
 <Player('Tian', '33')>,
 <Player('Masanori', '19')>]

In [34]:
# Assign Peter to Duke
peter.team = duke

In [35]:
# Assign Irene to Duke
players[2].team = duke
players[2].team

<team('Duke')>

In [36]:
players[3].team = t1
players[4].team = duke
players[5].team = duke
players[6].team = t1

In [37]:
# Check Duke's players
duke.players

[<Player('Peter Bachman', '5')>,
 <Player('Irene', '7')>,
 <Player('Leticia', '21')>,
 <Player('Tian', '33')>]

In [38]:
t1.players

[<Player('Cecilia', '24')>, <Player('Jie', '9')>, <Player('Masanori', '19')>]

Let's check the players team!

In [39]:
peter.team
# or
# players[0].team

<team('Duke')>

In [40]:
# We can get the roster from mason's team
peter.team.players

[<Player('Peter Bachman', '5')>,
 <Player('Irene', '7')>,
 <Player('Leticia', '21')>,
 <Player('Tian', '33')>]

In [41]:
# Now note the id:
peter.team_id

2

In [42]:
# Let's commit the changes! 
session.commit()

Lets load the two tables together using: 

`query(Table1).join(Table2)`

In [43]:
for i in session.query(Player).join(Team).filter(Player.name == "Cecilia").filter(Team.name == "WashU").order_by(Player.number):
  print(i.number, i.name, i.team.name)

24 Cecilia WashU


In [44]:
# equivalently using the and_()
for i in session.query(Player).join(Team).filter(and_(Player.name=="Cecilia", Team.name=="WashU")).order_by(Player.number):
  print(i.number, i.name, i.team.name)

24 Cecilia WashU


In [45]:
# now deletion
# list we queried above
players

[<Player('Peter Bachman', '5')>,
 <Player('Cecilia', '24')>,
 <Player('Irene', '7')>,
 <Player('Jie', '9')>,
 <Player('Leticia', '21')>,
 <Player('Tian', '33')>,
 <Player('Masanori', '19')>]

In [46]:
session.query(Player).filter(Player.number == 24).count()

1

In [47]:
# query 1 player
# now we have the representation of him
cecilia = session.query(Player).filter(Player.number == 24).first()

In [48]:
# now we can delete cecilia
session.delete(cecilia)

In [49]:
# it's gone!
session.query(Player).filter(Player.number == 24).count()

0

In [50]:
# But I am still in our object players
players

[<Player('Peter Bachman', '5')>,
 <Player('Cecilia', '24')>,
 <Player('Irene', '7')>,
 <Player('Jie', '9')>,
 <Player('Leticia', '21')>,
 <Player('Tian', '33')>,
 <Player('Masanori', '19')>]

In [51]:
# Let's update our players variable
players = session.query(Player).all()

In [52]:
players

[<Player('Peter Bachman', '5')>,
 <Player('Irene', '7')>,
 <Player('Jie', '9')>,
 <Player('Leticia', '21')>,
 <Player('Tian', '33')>,
 <Player('Masanori', '19')>]

In [53]:
# commit our changes
session.commit()

In [54]:
# print IDs 
[p.id for p in players]

[1, 3, 4, 5, 6, 7]

In [55]:
# Add Cecilia again
cecilia = Player(name = "Cecilia", number = 9)
session.add(cecilia)

In [56]:
session.commit()

In [57]:
players = session.query(Player).all()

In [58]:
players

[<Player('Peter Bachman', '5')>,
 <Player('Irene', '7')>,
 <Player('Jie', '9')>,
 <Player('Leticia', '21')>,
 <Player('Tian', '33')>,
 <Player('Masanori', '19')>,
 <Player('Cecilia', '9')>]

In [62]:
# How to convert data to csv
for player in players:
  ## apply skills we've learned already
  print(player.name, player.number, player.team, player.id)

Peter Bachman 5 <team('Duke')> 1
Irene 7 <team('Duke')> 3
Jie 9 <team('WashU')> 4
Leticia 21 <team('Duke')> 5
Tian 33 <team('Duke')> 6
Masanori 19 <team('WashU')> 7
Cecilia 9 None 8


In [63]:
# A more complete Example:
import csv
with open("players.csv", 'w') as f:
    my_writer = csv.DictWriter(f, fieldnames = ("name", "number", "team"))
    my_writer.writeheader()
    for player in players:
        try:
            my_writer.writerow({"name":player.name, "number":player.number, "team":player.team.name})
        except AttributeError:
            my_writer.writerow({"name":player.name, "number":player.number, "team":""})

In [61]:
# Copyright of the original version:

# Copyright (c) 2014 Matt Dickenson
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.